In [162]:
import pandas as pd
import numpy as np
import requests
from datetime import datetime, timedelta
import pytz
import json
import ta_formulas as ta
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

In [99]:
test_data = [{'contract_ticker': 'O:GOOGL240503C00167500', 'quantity': 2, 'last_price': 0.73, 'volume': 24599, 'spread_position': 0},
             {'contract_ticker': 'O:GOOGL240503C00170000', 'quantity': 5, 'last_price': 0.32, 'volume': 20319, 'spread_position': 1}]
test_option1 = test_data[0]['contract_ticker']
test_contracts1 = test_data[0]['quantity']
test_ticker1 = 'GOOGL'
start_date_str = '2024-04-30'
# to_stamp = '2024-05-01'
open_date = '2024-04-30 15:00'
key = "A_vXSwpuQ4hyNRj_8Rlw1WwVDWGgHbjp"
multiplier = 15
timespan = 'minute'

start_date = datetime.strptime(start_date_str, '%Y-%m-%d')
weekday = start_date.weekday()
days_till_friday = (4 - weekday) % 7
end_date = start_date + timedelta(days = days_till_friday)
end_date_str = end_date.strftime('%Y-%m-%d')


In [100]:
def convert_timestamp_est(timestamp):
    # Create a UTC datetime object from the timestamp
    utc_datetime = datetime.fromtimestamp(timestamp).replace(tzinfo=pytz.utc)
    # Define the EST timezone
    est_timezone = pytz.timezone('America/New_York')
    # Convert the UTC datetime to EST
    est_datetime = utc_datetime.astimezone(est_timezone)
    return est_datetime

def polygon_call_stocks(contract, from_stamp, to_stamp, multiplier, timespan, open_date):
    trading_hours = ['09:30:00', '16:00:00']
    try:
        payload={}
        headers = {}
        url = f"https://api.polygon.io/v2/aggs/ticker/{contract}/range/{multiplier}/{timespan}/{from_stamp}/{to_stamp}?adjusted=true&sort=asc&limit=50000&apiKey={key}"
        response = requests.request("GET", url, headers=headers, data=payload)
        res_df = pd.DataFrame(json.loads(response.text)['results'])
        res_df['t'] = res_df['t'].apply(lambda x: int(x/1000))
        res_df['date'] = res_df['t'].apply(lambda x: convert_timestamp_est(x))
        res_df['year'] = res_df['date'].apply(lambda x: x.year)
        res_df['month'] = res_df['date'].apply(lambda x: x.month)
        res_df['time'] = res_df['date'].apply(lambda x: x.time())
        res_df['hour'] = res_df['date'].apply(lambda x: x.hour)
        res_df['minute'] = res_df['date'].apply(lambda x: x.minute)
        res_df['dt'] = res_df['date'].apply(lambda x: datetime(x.year, x.month, x.day,x.hour,x.minute))
        res_df = res_df[(res_df['time'] >= datetime.strptime(trading_hours[0], '%H:%M:%S').time()) & (res_df['time'] <= datetime.strptime(trading_hours[1], '%H:%M:%S').time())]
        res_df = res_df[res_df['dt'] >= datetime.strptime(open_date, '%Y-%m-%d %H:%M')]

        res_df.drop(['vw', 'h', 'l', 'c', 'n', 't','date', 'year','month', 'time', 'hour', 'minute'], axis = 1, inplace = True)
        res_df.reset_index(inplace=True)
        res_df.rename(columns = {'v': 'underlying_volume', 'o': 'underlying_price'}, inplace=True)
        res_df['symbol'] = contract
        res_df = res_df[['symbol', 'dt', 'underlying_price', 'underlying_volume']]

            
        return res_df
    except Exception as e:  
        print(e)
        return pd.DataFrame()

def polygon_call_options(contract, from_stamp, to_stamp, multiplier, timespan, open_date):
    trading_hours = ['09:30:00', '16:00:00']
    try:
        payload={}
        headers = {}
        url = f"https://api.polygon.io/v2/aggs/ticker/{contract}/range/{multiplier}/{timespan}/{from_stamp}/{to_stamp}?adjusted=true&sort=asc&limit=50000&apiKey={key}"
        response = requests.request("GET", url, headers=headers, data=payload)
        res_df = pd.DataFrame(json.loads(response.text)['results'])
        res_df['t'] = res_df['t'].apply(lambda x: int(x/1000))
        res_df['date'] = res_df['t'].apply(lambda x: convert_timestamp_est(x))
        res_df['year'] = res_df['date'].apply(lambda x: x.year)
        res_df['month'] = res_df['date'].apply(lambda x: x.month)
        res_df['time'] = res_df['date'].apply(lambda x: x.time())
        res_df['hour'] = res_df['date'].apply(lambda x: x.hour)
        res_df['minute'] = res_df['date'].apply(lambda x: x.minute)
        res_df['dt'] = res_df['date'].apply(lambda x: datetime(x.year, x.month, x.day,x.hour,x.minute))
        res_df = res_df[(res_df['time'] >= datetime.strptime(trading_hours[0], '%H:%M:%S').time()) & (res_df['time'] <= datetime.strptime(trading_hours[1], '%H:%M:%S').time())]
        res_df = res_df[res_df['dt'] >= datetime.strptime(open_date, '%Y-%m-%d %H:%M')]

        res_df.drop(['vw', 'h', 'l', 'c', 'n', 't','date', 'year','month', 'time', 'hour', 'minute'], axis = 1, inplace = True)
        res_df.reset_index(inplace=True)
        res_df.rename(columns = {'v': 'option_volume', 'o': 'option_price'}, inplace=True)
        res_df['option_contract'] = contract
        res_df = res_df[['option_contract', 'dt', 'option_price', 'option_volume']]
            
        return res_df
    except Exception as e:  
        print(e)
        return pd.DataFrame()

In [101]:
class CustomRetry(Retry):
    def is_retry(self, method, status_code, has_retry_after=False):
        """ Return True if we should retry the request, otherwise False. """
        if status_code != 200:
            return True
        return super().is_retry(method, status_code, has_retry_after)
    
def setup_session_retries(
    retries: int = 3,
    backoff_factor: float = 0.05,
    status_forcelist: tuple = (500, 502, 504),
):
    """
    Sets up a requests Session with retries.
    
    Parameters:
    - retries: Number of retries before giving up. Default is 3.
    - backoff_factor: A factor to use for exponential backoff. Default is 0.3.
    - status_forcelist: A tuple of HTTP status codes that should trigger a retry. Default is (500, 502, 504).

    Returns:
    - A requests Session object with retry configuration.
    """
    retry_strategy = CustomRetry(
        total=retries,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist,
        allowed_methods=frozenset(["GET", "POST", "PUT", "DELETE", "HEAD", "OPTIONS"]),
        raise_on_status=False
    )
    adapter = HTTPAdapter(max_retries=retry_strategy)
    session = requests.Session()
    session.mount("http://", adapter)
    session.mount("https://", adapter)
    
    return session

def execute_polygon_call(url):
    session = setup_session_retries()
    response = session.request("GET", url, headers={}, data={})
    return response 
    
def stat_window_creator(symbol, from_stamp, to_stamp, timespan, multiplier):
    trading_hours = [9,10,11,12,13,14,15]
    
    from_dt = datetime.strptime(from_stamp, '%Y-%m-%d')
    new_stamp = from_dt - timedelta(days = 1)
    from_stamp = new_stamp.strftime('%Y-%m-%d')

    data = []
    url = f"https://api.polygon.io/v2/aggs/ticker/{symbol}/range/{multiplier}/{timespan}/{from_stamp}/{to_stamp}?adjusted=true&sort=asc&limit=50000&apiKey={key}"
    response = execute_polygon_call(url)
    response_data = json.loads(response.text)
    results = response_data['results']
    results_df = pd.DataFrame(results)
    results_df['t'] = results_df['t'].apply(lambda x: int(x/1000))
    results_df['date'] = results_df['t'].apply(lambda x: convert_timestamp_est(x))
    results_df['hour'] = results_df['date'].apply(lambda x: x.hour)
    results_df['minute'] = results_df['date'].apply(lambda x: x.minute)
    results_df['day'] = results_df['date'].apply(lambda x: x.day)
    results_df['month'] = results_df['date'].apply(lambda x: x.month)
    results_df['symbol'] = symbol
    trimmed_df = results_df.loc[results_df['hour'].isin(trading_hours)]
    filtered_df = trimmed_df.loc[~((trimmed_df['hour'] == 9) & (trimmed_df['minute'] < 30))]

    return filtered_df



In [102]:
df = polygon_call_stocks(test_ticker1, start_date_str, end_date_str, multiplier, timespan, open_date)
df2 = polygon_call_options(test_ticker1, start_date_str, end_date_str, multiplier, timespan, open_date)
base_df = pd.merge(df, df2, on='dt')
base_df = base_df[['symbol', 'option_contract', 'dt', 'underlying_price', 'underlying_volume', 'option_price', 'option_volume']]
base_df.head(10)


,symbol,option_contract,dt,underlying_price,underlying_volume,option_price,option_volume
0,GOOGL,GOOGL,2024-04-30 15:00:00,163.1600,3042.0,163.1600,3042.0
1,GOOGL,GOOGL,2024-04-30 15:15:00,163.1900,4422.0,163.1900,4422.0
2,GOOGL,GOOGL,2024-04-30 15:30:00,162.9800,3341.0,162.9800,3341.0
3,GOOGL,GOOGL,2024-04-30 15:45:00,162.8512,8677.0,162.8512,8677.0
4,GOOGL,GOOGL,2024-05-01 09:30:00,164.2550,643019.0,164.2550,643019.0
5,GOOGL,GOOGL,2024-05-01 09:45:00,164.2850,835351.0,164.2850,835351.0
6,GOOGL,GOOGL,2024-05-01 10:00:00,164.8900,1484459.0,164.8900,1484459.0
7,GOOGL,GOOGL,2024-05-01 10:15:00,165.0336,935223.0,165.0336,935223.0
8,GOOGL,GOOGL,2024-05-01 10:30:00,165.3200,1783556.0,165.3200,1783556.0
9,GOOGL,GOOGL,2024-05-01 10:45:00,166.2900,1102466.0,166.2900,1102466.0


In [186]:
base_df['contracts'] = test_contracts1
# init_cost = base_df['option_price'][0] * base_df['contracts'][0]
# base_df['initial_cost'] = init_cost
base_df['contract_value'] = base_df['option_price'].apply(lambda x: (x * base_df['contracts'][0]).round(2))
base_df.head(10)

,symbol,option_contract,dt,underlying_price,underlying_volume,option_price,option_volume,contracts,contract_value
0,GOOGL,GOOGL,2024-04-30 15:00:00,163.1600,3042.0,163.1600,3042.0,2,326.32
1,GOOGL,GOOGL,2024-04-30 15:15:00,163.1900,4422.0,163.1900,4422.0,2,326.38
2,GOOGL,GOOGL,2024-04-30 15:30:00,162.9800,3341.0,162.9800,3341.0,2,325.96
3,GOOGL,GOOGL,2024-04-30 15:45:00,162.8512,8677.0,162.8512,8677.0,2,325.70
4,GOOGL,GOOGL,2024-05-01 09:30:00,164.2550,643019.0,164.2550,643019.0,2,328.51
5,GOOGL,GOOGL,2024-05-01 09:45:00,164.2850,835351.0,164.2850,835351.0,2,328.57
6,GOOGL,GOOGL,2024-05-01 10:00:00,164.8900,1484459.0,164.8900,1484459.0,2,329.78
7,GOOGL,GOOGL,2024-05-01 10:15:00,165.0336,935223.0,165.0336,935223.0,2,330.07
8,GOOGL,GOOGL,2024-05-01 10:30:00,165.3200,1783556.0,165.3200,1783556.0,2,330.64
9,GOOGL,GOOGL,2024-05-01 10:45:00,166.2900,1102466.0,166.2900,1102466.0,2,332.58


In [195]:
# for i, row in base_df.iterrows():
#     window_df = stat_window_creator(base_df['symbol'][i], start_date_str, end_date_str, timespan, multiplier)
#     print(window_df)
    # rsi = rsi(window_df['c'], 15)
    # roc = roc(window_df['c'], 15)
    # print(rsi)

window_df = stat_window_creator(base_df['symbol'][0], start_date_str, end_date_str, timespan, multiplier)
window_df['dt'] = window_df['date'].apply(lambda x: x.tz_localize(None))
print(window_df)
window_df.reset_index(drop = True, inplace=True)
window_df.set_index('date',inplace=True)

rsi = ta.rsi(window_df['c'], window=15)
# rsi.head(1000)
roc = ta.roc(window_df['c'], window=15)

# final_df = pd.merge(base_df, rsi, left_on='dt', right_on='date')


# window_df.head()

<Response [200]>
